# Hemalatha Velappan: Classification of different tree species plantations using deep learning

[Video recording](https://youtu.be/1OAzeb71lwU)

### The goal of this work is to develop a model to identify planted forests and the tree species growing there. The model is developed using the 
#### (1) known locations of planted forests based on literature and personal communications, 
#### (2) image analysis and feature extraction of planted trees
#### (3) spectral signatures unique to each species

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy
from osgeo import ogr
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [52]:
#Loading plantation shapefile location

%cd /media/sf_LVM_shared/my_SE_data/Plantation_datasets/Peru_Plantation_Shapefile-Updated

/media/sf_LVM_shared/my_SE_data/Plantation_datasets/Peru_Plantation_Shapefile-Updated


In [9]:
!gdalinfo -mm SentinelMap.tif

Driver: GTiff/GeoTIFF
Files: SentinelMap.tif
Size is 898, 770
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-76.889769608227439,-7.186342609899349)
Pixel Size = (0.000269494585236,-0.000269494585236)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  ( -76.8897696,  -7.1863426) ( 76d53'23.17"W,  7d11'10.83"S)
Lower Left  ( -76.8897696,  -7.3938534) ( 76d53'23.17"W,  7d23'37.87"S)
Upper Right ( -

## Performing zonal statistics on the polygon shapefile wrt the sentinel satellite image

In [3]:
!pkextractogr -f CSV -i SentinelMap.tif -s Peru_XY/Peru_with_XY.shp -r allpoints  -r mean -r stdev -o extracted2.csv

processing layer Peru_with_XY
0...10...20...30...40...50...60...70...80...90...100 - done.


In [3]:
predictors = pd.read_csv("extracted2.csv")
predictors.head(10)

OBJECTID       FUENTE/SOU    DOCREG   FECREG  \
0     99039  GORE San Martín  APP-RNPF  27:59.0   
1     99039  GORE San Martín  APP-RNPF  27:59.0   
2     99039  GORE San Martín  APP-RNPF  27:59.0   
3     99039  GORE San Martín  APP-RNPF  27:59.0   
4     99039  GORE San Martín  APP-RNPF  27:59.0   
5     99039  GORE San Martín  APP-RNPF  27:59.0   
6     99039  GORE San Martín  APP-RNPF  27:59.0   
7     99039  GORE San Martín  APP-RNPF  27:59.0   
8     99039  GORE San Martín  APP-RNPF  27:59.0   
9     99039  GORE San Martín  APP-RNPF  27:59.0   

                                              OBSERV  ZONUTM  ORIGEN  TIPCOM  \
0  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
1  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
2  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
3  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
4  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
5  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
6  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
7  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
8  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   
9  ara autorizar el aprovechamiento la U.O.G.F.  ...      18       1       1   

                    NUMREG                NOMTIT/Tit  ...   b4    b5    b6  \
0  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  798  1965  2724   
1  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  796  2069  2740   
2  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  805  2114  2717   
3  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  741  1731  2253   
4  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  854  2127  2660   
5  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  756  1901  2482   
6  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  739  1792  2374   
7  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  797  2108  2745   
8  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  738  1755  2448   
9  22-SAM/REG-PLT-2019-050  CARBAJAL VIGO, SEBASTIAN  ...  817  1925  2571   

     b7    b8   b9  b10   b11  b12 Classification  
0  2588  2981  475   11  1301  493             10  
1  2661  3053  475   11  1269  460             10  
2  2656  3130  447   10  1309  485             10  
3  2031  2501  447   10  1197  440             10  
4  2651  3144  461    9  1496  580             10  
5  2427  2841  461    9  1302  492             10  
6  2293  2648  438    9  1283  518             10  
7  2731  3185  488   10  1364  502             10  
8  2214  2763  472    9  1178  439             10  
9  2396  2900  472    9  1404  533             10  

[10 rows x 56 columns]

## The following are the tree species and the corresponding label numbers

##### Acrocarpus fraxinifolius	1
##### Calycophyllum spruceanum	2
##### Cedrela Mixed	3
##### Guazuma crinita	4
##### Miconia barbeyana	5
##### Ochroma pyramidale	6
##### Other Mixed	7
##### Swietenia Cedrela Mixed	8
##### Swietenia macrophylla	9
##### Swietenia Mixed	10

In [4]:
Desired_columns = ['X', 'Y', 'b0', 'b1','b2', 'b3','b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'Classification']
print(Desired_columns)

['X', 'Y', 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'Classification']


In [5]:
Desired_Output = predictors[Desired_columns]
Desired_Output.head()

X       Y    b0   b1   b2   b3   b4    b5    b6    b7    b8   b9  \
0 -76.75757 -7.1862  1230  932  792  507  798  1965  2724  2588  2981  475   
1 -76.75757 -7.1862  1230  922  787  479  796  2069  2740  2661  3053  475   
2 -76.75757 -7.1862  1227  930  806  495  805  2114  2717  2656  3130  447   
3 -76.75757 -7.1862  1227  916  761  476  741  1731  2253  2031  2501  447   
4 -76.75757 -7.1862  1232  943  823  535  854  2127  2660  2651  3144  461   

   b10   b11  b12  Classification  
0   11  1301  493              10  
1   11  1269  460              10  
2   10  1309  485              10  
3   10  1197  440              10  
4    9  1496  580              10

In [6]:
Desired_Output = Desired_Output.to_numpy()

## The input and output variables are split between training and testing by 70:30

#### All the 14 columns are X variables. The final column that has categorical numbers is the target or Y variable

In [7]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(Desired_Output[:,:14], Desired_Output[:,15], test_size=0.30, random_state=0)
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)-1
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)-1
print('X_train.shape: {}, X_test.shape: {}, y_train.shape: {}, y_test.shape: {}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train.shape: torch.Size([6942, 14]), X_test.shape: torch.Size([2976, 14]), y_train.shape: torch.Size([6942]), y_test.shape: torch.Size([2976])


## The feedforward module with 3 hidden layers are built with different activation functions applied to the layers

In [8]:
# Creating a feedforward module

class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size=10):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(self.hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh = torch.nn.Tanh()
    def forward(self, x):
        hidden = self.relu(self.fc1(x))
        hidden = self.relu(self.fc2(hidden))
        output = self.tanh(self.fc3(hidden))

        return output

In [9]:
model = Feedforward(14, 256) #input_size = 14 and hidden_size = 256
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [10]:
epochs = 27 #27 is chosen because choosing higher numbers make python to crash and after 26 the loss is stabilized
aggregated_losses = []

for i in range(epochs):
    i += 1
    y_pred = model(X_train)
    single_loss = loss_function(y_pred, y_train)
    aggregated_losses.append(single_loss)

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 2.96000528
epoch:  26 loss: 2.80332470
epoch:  27 loss: 2.8033246994


In [11]:
with torch.no_grad():
    y_val = model(X_test)
    loss = loss_function(y_val, y_test)
print(f'Loss: {loss:.8f}')

Loss: 2.82178712


In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_val.argmax(dim=1)))
print(classification_report(y_test,y_val.argmax(dim=1)))
print(accuracy_score(y_test, y_val.argmax(dim=1)))

[[  0  52   0   0   0   0   0   0   0]
 [  0  17   0   0   0   0   0   0   0]
 [  0  10   0   0   0   0   0   0   0]
 [  0 269   0   0   0   0   0   0   0]
 [  0 869   0   0   0   0   0   0   0]
 [  0 922   0   0   0   0   0   0   0]
 [  0 293   0   0   0   0   0   0   0]
 [  0   8   0   0   0   0   0   0   0]
 [  0 536   0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        52
           1       0.01      1.00      0.01        17
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00       269
           5       0.00      0.00      0.00       869
           6       0.00      0.00      0.00       922
           7       0.00      0.00      0.00       293
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00       536

    accuracy                           0.01      2976
   macro avg       0.00      0.11      0.00      297

/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Creating a single-layer perceptron model

In [13]:
# Create the model
class Perceptron(torch.nn.Module):
    def __init__(self,input_size, output_size,use_activation_fn=None):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size,output_size)
        self.relu = torch.nn.ReLU() # instead of Heaviside step fn
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh = torch.nn.Tanh()
        self.use_activation_fn=use_activation_fn
    def forward(self, x):
        output = self.fc(x)
        if self.use_activation_fn=='sigmoid':
            output = self.sigmoid(output) # To add the non-linearity. Try training you Perceptron with and without the non-linearity
        elif self.use_activation_fn=='tanh':
            output = self.tanh(output) 
        elif self.use_activation_fn=='relu':
            output = self.relu(output) 

        return output

In [46]:
model = Perceptron(input_size=14, output_size=10, use_activation_fn='tanh')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [47]:
epochs = 300
aggregated_losses = []

for i in range(epochs):
    i += 1
    y_pred = model(X_train)
    single_loss = loss_function(y_pred, y_train)
    aggregated_losses.append(single_loss)

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 2.86867380
epoch:  26 loss: 2.86806893
epoch:  51 loss: 2.86806870
epoch:  76 loss: 2.86806870
epoch: 101 loss: 2.86806870
epoch: 126 loss: 2.86806870
epoch: 151 loss: 2.86801815
epoch: 176 loss: 2.86790514
epoch: 201 loss: 2.86790514
epoch: 226 loss: 2.86790514
epoch: 251 loss: 2.86790514
epoch: 276 loss: 2.86790514
epoch: 300 loss: 2.8679051399


In [48]:
with torch.no_grad():
    y_val = model(X_test)
    loss = loss_function(y_val, y_test)
print(f'Loss: {loss:.8f}')

Loss: 2.87435365


In [49]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_val.argmax(dim=1)))
print(classification_report(y_test,y_val.argmax(dim=1)))
print(accuracy_score(y_test, y_val.argmax(dim=1)))

[[ 52   0   0   0   0   0   0   0   0]
 [ 17   0   0   0   0   0   0   0   0]
 [ 10   0   0   0   0   0   0   0   0]
 [269   0   0   0   0   0   0   0   0]
 [869   0   0   0   0   0   0   0   0]
 [922   0   0   0   0   0   0   0   0]
 [293   0   0   0   0   0   0   0   0]
 [  8   0   0   0   0   0   0   0   0]
 [536   0   0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

           0       0.02      1.00      0.03        52
           1       0.00      0.00      0.00        17
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00       269
           5       0.00      0.00      0.00       869
           6       0.00      0.00      0.00       922
           7       0.00      0.00      0.00       293
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00       536

    accuracy                           0.02      2976
   macro avg       0.00      0.11      0.00      297

### Results:

#### Using single-layer perceptron and multi-layer feedforward deep learning models did not yield high accuracy in distinguishing different tree plantations using Sentinel-2 dataset. There could be multiple reasons behind this. Since spectral band information from Sentinel 2 satellite image constitute most of the X variables and because of the high spectral similarity between tree species, the model couldn't find a way to distinguish the species. It can also because of the poor modeling parameters such as poor selection of activation functions, learning rate, epoch numbers etc. which could have affected the model as well. In the future, more input parameters like texture metrics, band information from sentinel-1 etc. can be added into the model for better variance. Further different modelling parameters can be explored to improve the accuracy.

In [55]:
!jupyter nbconvert --to html /media/sf_LVM_shared/my_SE_data/exercise/Final_Project.ipynb

[NbConvertApp] Converting notebook /media/sf_LVM_shared/my_SE_data/exercise/Final_Project.ipynb to html
[NbConvertApp] Writing 642237 bytes to /media/sf_LVM_shared/my_SE_data/exercise/Final_Project.html
